# Exploratory Data Analysis with Pyspark and Spark SQL

The following notebook utilizes New York City taxi data from [TLC Trip Record Data](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

## Instructions

- Load and explore nyc taxi data from january 0f 2019. The exercises can be executed using pyspark or spark sql ( a subset of the questions will be re-answered using the language not chosen for the  main work).
- Load the zone lookup table to answer the questions about the nyc boroughs.  
- Load nyc taxi data from January of 2025 and compare data.  
- With any remaining time, work on the where to go from here section.  
- Lab due date is TBD ( due dates will be updated in the readme for the class repo )

In [0]:
# Define the name of the new catalog
catalog = 'taxi_eda_db'

# define variables for the trips data
schema = 'yellow_taxi_trips'
volume = 'data'
file_name = 'yellow_tripdata_2019-01.parquet'
table_name = 'tbl_yellow_taxi_trips'
path_volume = '/Volumes/' + catalog + "/" + schema + '/' + volume
path_table =  catalog + "." + schema
download_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2019-01.parquet'

In [0]:
# create the catalog/schema/volume
spark.sql('create catalog if not exists ' + catalog)
spark.sql('create schema if not exists ' + catalog + '.' + schema)
spark.sql('create volume if not exists ' + catalog + '.' + schema + '.' + volume)

DataFrame[]

In [0]:
# Get the data
dbutils.fs.cp(f"{download_url}", f"{path_volume}" + "/" + f"{file_name}")

True

In [0]:
# create the dataframe
df_trips = spark.read.parquet(f"{path_volume}/{file_name}",
  header=True,
  inferSchema=True,
  sep=",")

In [0]:
# Show the dataframe
df_trips.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2019-01-01 00:46:40|  2019-01-01 00:53:20|            1.0|          1.5|       1.0|                 N|         151|         239|           1|        7.0|  0.5|    0.5|      1.6

## Lab

### Part 1
This section can be completed either using pyspark commands or sql commands ( There will be a section after in which a self-chosen subset of the questions are re-answered using the language not used for the main section. i.e. if pyspark is chosen for the main lab, sql should be used to repeat some of the questions. )

- Add a column that creates a unique key to identify each record in order to answer questions about individual trips
- Which trip has the highest passanger count
- What is the Average passanger count
- Shortest/longest trip by distance? by time?.
- busiest day/slowest single day
- busiest/slowest time of day ( you may want to bucket these by hour or create timess such as morning, afternoon, evening, late night )
- On average which day of the week is slowest/busiest
- Does trip distance or num passangers affect tip amount
- What was the highest "extra" charge and which trip
- Are there any datapoints that seem to be strange/outliers (make sure to explain your reasoning in a markdown cell)?

##### 1. Add a column that creates a unique key to identify each record in order to answer questions about individual trips


In [0]:
from pyspark.sql.functions import monotonically_increasing_id

# Add a new column that gives each trip a unique ID
df_trips = df_trips.withColumn("unique_id", monotonically_increasing_id())

# Show first 5 rows to confirm
df_trips.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|unique_id|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+
|       1| 2019-01-01 00:46:40|  2019-01-01 00:53:20|            1.0|          1.5|       1.0|                 N|         151|         239|           1|     

We added a new column called unique_id. This gives each trip a different number so that we can identify them easily.

##### 2. Which trip has the highest passanger count


In [0]:
from pyspark.sql.functions import col

# Find the trip(s) with the most passengers
max_passenger = df_trips.agg({"passenger_count": "max"}).collect()[0][0]

df_highest_passenger = df_trips.filter(col("passenger_count") == max_passenger)

df_highest_passenger.select("unique_id", "passenger_count", "trip_distance", "total_amount").show()

+---------+---------------+-------------+------------+
|unique_id|passenger_count|trip_distance|total_amount|
+---------+---------------+-------------+------------+
|   949956|            9.0|          0.0|        12.6|
|  1296287|            9.0|          0.0|         9.3|
|  2012098|            9.0|          0.0|        11.3|
|  2883995|            9.0|          0.0|       12.25|
|  4534707|            9.0|          0.0|      110.76|
|  4852225|            9.0|          0.0|       12.74|
|  4997790|            9.0|          0.0|         9.8|
|  7286683|            9.0|          0.0|        10.3|
|  7373876|            9.0|        13.38|        90.8|
+---------+---------------+-------------+------------+



The largest passenger count we found is 9 passengers. Several trips show this value. Some records look suspicious because the distance is 0.0 but still have a fare. The most reasonable record is the trip with about 13 miles and $90 total, which matches a real group taxi ride.

##### 3. What is the Average passanger count


In [0]:
from pyspark.sql.functions import avg

# Calculate the average number of passengers per trip
avg_passenger = df_trips.agg(avg("passenger_count").alias("avg_passenger_count"))

avg_passenger.show()

+-------------------+
|avg_passenger_count|
+-------------------+
| 1.5670317144945614|
+-------------------+



On average, there are about 1.6 passengers per trip. This result is reasonable because most rides are just one person traveling alone, while fewer rides include two or more people.

##### 4a. Shortest/longest trip by distance?

In [0]:
from pyspark.sql.functions import col

# Shortest trip by distance
shortest_trip = df_trips.orderBy(col("trip_distance").asc()).limit(1)

# Longest trip by distance
longest_trip = df_trips.orderBy(col("trip_distance").desc()).limit(1)

print("Shortest trip by distance:")
shortest_trip.select("unique_id", "passenger_count", "trip_distance").show()

print("Longest trip by distance:")
longest_trip.select("unique_id", "passenger_count", "trip_distance").show()


Shortest trip by distance:
+---------+---------------+-------------+
|unique_id|passenger_count|trip_distance|
+---------+---------------+-------------+
|        2|            3.0|          0.0|
+---------+---------------+-------------+

Longest trip by distance:
+---------+---------------+-------------+
|unique_id|passenger_count|trip_distance|
+---------+---------------+-------------+
|  6074091|            1.0|        831.8|
+---------+---------------+-------------+



The shortest trip has a distance of 0 miles, which doesn’t make sense for a real ride (probably a data issue). The longest trip shows 831 miles, which is also not realistic for NYC. These are examples of outliers in the dataset. Most trips should be just a few miles long.

##### 4b. Shortest/longest trip by time?

In [0]:
from pyspark.sql.functions import unix_timestamp

# Add a new column for trip duration in minutes
df_trips = df_trips.withColumn(
    "trip_duration_min",
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60
)

# Shortest trip by time
shortest_time = df_trips.orderBy(col("trip_duration_min").asc()).limit(1)

# Longest trip by time
longest_time = df_trips.orderBy(col("trip_duration_min").desc()).limit(1)

print("Shortest trip by time:")
shortest_time.select("unique_id", "passenger_count", "trip_duration_min").show()

print("Longest trip by time:")
longest_time.select("unique_id", "passenger_count", "trip_duration_min").show()


Shortest trip by time:
+---------+---------------+-----------------+
|unique_id|passenger_count|trip_duration_min|
+---------+---------------+-----------------+
|  1203184|            1.0|         -84280.5|
+---------+---------------+-----------------+

Longest trip by time:
+---------+---------------+-----------------+
|unique_id|passenger_count|trip_duration_min|
+---------+---------------+-----------------+
|    68267|            1.0|43648.01666666667|
+---------+---------------+-----------------+



The shortest trip shows a negative duration, which means the drop-off time was recorded before the pickup time. The longest trip lasted more than 30 days, which is not possible for a taxi ride. Both are data errors/outliers. Realistic trips should normally last a few minutes to maybe a couple of hours.

##### 5. busiest day/slowest single day

In [0]:
from pyspark.sql.functions import to_date, count

# Extract only the date (without time)
df_trips = df_trips.withColumn("trip_date", to_date("tpep_pickup_datetime"))

# Count trips per day
trips_per_day = df_trips.groupBy("trip_date").agg(count("*").alias("num_trips"))

# Find busiest day (max trips) and slowest day (min trips)
busiest_day = trips_per_day.orderBy(col("num_trips").desc()).limit(1)
slowest_day = trips_per_day.orderBy(col("num_trips").asc()).limit(1)

print("Busiest day:")
busiest_day.show()

print("Slowest day:")
slowest_day.show()


Busiest day:
+----------+---------+
| trip_date|num_trips|
+----------+---------+
|2019-01-25|   292499|
+----------+---------+

Slowest day:
+----------+---------+
| trip_date|num_trips|
+----------+---------+
|2018-12-21|        1|
+----------+---------+



The busiest day in our data was January 25, 2019, with almost 300k trips. This makes sense for a busy city like New York. The slowest day was December 21, 2018, with only 1 trip, which is not realistic. That means the data for this day is incomplete or corrupted.

##### 6. busiest/slowest time of day ( you may want to bucket these by hour or create timess such as morning, afternoon, evening, late night )

In [0]:
from pyspark.sql.functions import hour

# Extract the hour from pickup time
df_trips = df_trips.withColumn("pickup_hour", hour("tpep_pickup_datetime"))

# Count trips per hour
trips_per_hour = df_trips.groupBy("pickup_hour").agg(count("*").alias("num_trips"))

# Find busiest and slowest hour
busiest_hour = trips_per_hour.orderBy(col("num_trips").desc()).limit(1)
slowest_hour = trips_per_hour.orderBy(col("num_trips").asc()).limit(1)

print("Busiest hour of the day:")
busiest_hour.show()

print("Slowest hour of the day:")
slowest_hour.show()


Busiest hour of the day:
+-----------+---------+
|pickup_hour|num_trips|
+-----------+---------+
|         18|   515390|
+-----------+---------+

Slowest hour of the day:
+-----------+---------+
|pickup_hour|num_trips|
+-----------+---------+
|          4|    61424|
+-----------+---------+



The busiest time of day is around 6 PM, which matches the evening rush when many people leave work. The slowest time is around 4 AM, which is expected because the city is much quieter at night.

##### 7. On average which day of the week is slowest/busiest

In [0]:
from pyspark.sql.functions import dayofweek, date_format, count, col

# Add weekday number (1=Sunday ... 7=Saturday) and weekday name
df_trips = df_trips.withColumn("day_of_week_num", dayofweek("tpep_pickup_datetime"))
df_trips = df_trips.withColumn("day_of_week", date_format("tpep_pickup_datetime", "EEEE"))

# Count trips per day of week
trips_per_weekday = df_trips.groupBy("day_of_week_num", "day_of_week") \
    .agg(count("*").alias("num_trips")) \
    .orderBy(col("day_of_week_num").asc())

print("Trips per day of the week:")
trips_per_weekday.show()

# Find busiest and slowest day
busiest_weekday = trips_per_weekday.orderBy(col("num_trips").desc()).limit(1)
slowest_weekday = trips_per_weekday.orderBy(col("num_trips").asc()).limit(1)

print("Busiest weekday:")
busiest_weekday.show()

print("Slowest weekday:")
slowest_weekday.show()




Trips per day of the week:
+---------------+-----------+---------+
|day_of_week_num|day_of_week|num_trips|
+---------------+-----------+---------+
|              1|     Sunday|   859905|
|              2|     Monday|   908121|
|              3|    Tuesday|  1209084|
|              4|  Wednesday|  1265264|
|              5|   Thursday|  1357043|
|              6|     Friday|  1087215|
|              7|   Saturday|  1009985|
+---------------+-----------+---------+

Busiest weekday:
+---------------+-----------+---------+
|day_of_week_num|day_of_week|num_trips|
+---------------+-----------+---------+
|              5|   Thursday|  1357043|
+---------------+-----------+---------+

Slowest weekday:
+---------------+-----------+---------+
|day_of_week_num|day_of_week|num_trips|
+---------------+-----------+---------+
|              1|     Sunday|   859905|
+---------------+-----------+---------+



On average, Thursday is the busiest day of the week, while Sunday is the slowest. This matches real-world behavior: weekdays usually have more taxi demand due to work and travel, while Sundays are quieter since fewer people are commuting.

##### 8. Does trip distance or num passangers affect tip amount

In [0]:
from pyspark.sql.functions import corr

# Correlation between trip distance and tip amount
corr_distance_tip = df_trips.corr("trip_distance", "tip_amount")

# Correlation between passenger count and tip amount
corr_passenger_tip = df_trips.corr("passenger_count", "tip_amount")

print("Correlation (trip distance vs tip):", corr_distance_tip)
print("Correlation (passenger count vs tip):", corr_passenger_tip)


Correlation (trip distance vs tip): 0.5269200663652669
Correlation (passenger count vs tip): 0.004431051585116288


We looked at how trip distance and number of passengers affect the tip amount.
- The correlation between trip distance and tips is about 0.53, which is a moderate positive relationship. This means that longer trips usually get bigger tips.
- The correlation between passenger count and tips is almost zero (0.004), so the number of passengers doesn’t affect tips.

In short : tips depend more on distance, not on how many people are in the taxi.

##### 9. What was the highest "extra" charge and which trip

In [0]:
# Find the trip with the highest "extra" charge
highest_extra = df_trips.orderBy(col("extra").desc()).limit(1)

print("Trip with the highest extra charge:")
highest_extra.select("unique_id", "passenger_count", "trip_distance", "extra", "total_amount").show()


Trip with the highest extra charge:
+---------+---------------+-------------+------+------------+
|unique_id|passenger_count|trip_distance| extra|total_amount|
+---------+---------------+-------------+------+------------+
|  5323483|            1.0|          0.0|535.38|   356214.78|
+---------+---------------+-------------+------+------------+



We found that the trip with the highest "extra" charge had an extra fee of 535.38. However, the trip distance is 0.0, which suggests this is likely an outlier or error in the data (because it’s unrealistic to have such a huge charge for no distance traveled).

##### 10. Are there any datapoints that seem to be strange/outliers (make sure to explain your reasoning in a markdown cell)?

In [0]:
# Trips with 0 or negative distance
weird_distance = df_trips.filter(col("trip_distance") <= 0)

# Trips with negative fare or tip
weird_fares = df_trips.filter((col("fare_amount") < 0) | (col("tip_amount") < 0))

# Trips with very long duration (e.g. more than 5 hours)
weird_duration = df_trips.filter(col("trip_duration_min") > 300)

print("Trips with 0 or negative distance:")
weird_distance.show(5)

print("Trips with negative fare or tip:")
weird_fares.show(5)

print("Trips with very long duration:")
weird_duration.show(5)


Trips with 0 or negative distance:
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+------------------+----------+-----------+---------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|unique_id| trip_duration_min| trip_date|pickup_hour|day_of_week_num|day_of_week|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----

When we checked for unusual trips, we found :
- Some trips have 0 distance but still show a charge. These are probably errors or canceled rides.
- Some trips have negative fares or tips, which are not realistic and are likely mistakes in the data.
- A few trips have extremely long durations (over 26 hours), which is not possible for a taxi in New York.

These strange values are outliers. In real projects, we would usually clean the data by removing or correcting these records before doing analysis.

### Part 2

- Using the code for loading the first dataset as an example, load in the taxi zone lookup and answer the following questions
- which borough had most pickups? dropoffs?
- what are the busy/slow times by borough 
- what are the busiest days of the week by borough?
- what is the average trip distance by borough?
- what is the average trip fare by borough?
- highest/lowest faire amounts for a trip, what burough is associated with the each
- load the dataset from the most recently available january, is there a change to any of the average metrics.

In [0]:
# Define the variable for the taxi zone lookup
#catalog = 'taxi_eda_db'
schema2 = 'taxi_zone_lookups'
volume2 = 'data2'
table_name2 = 'tbl_taxi_zone_lookup'
file_name2 = 'file_taxi_zone_lookup'
path_volume2 = '/Volumes/' + catalog + "/" + schema2 + '/' + volume2
path_table2 =  catalog + "." + schema2
download_url2 = 'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv'

# create the catalog/schema/volume
spark.sql('create schema if not exists ' + catalog + '.' + schema2)
spark.sql('create volume if not exists ' + catalog + '.' + schema2 + '.' + volume2)
spark.sql(f"create table if not exists {catalog}.{schema2}.{table_name2} (LocationID INT, Borough STRING, Zone STRING, service_zone STRING)")

# Get the data
dbutils.fs.cp(f"{download_url2}", f"{path_volume2}" + "/" + f"{file_name2}")

True

In [0]:
df_trips.schema

StructType([StructField('VendorID', LongType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', DoubleType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', DoubleType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', IntegerType(), True), StructField('unique_id', LongT

1. Which borough as most lookup and dropoff ?

Manhattan has most pickup and dropoff (see df_borough table)

In [0]:
# look for the file
# display(dbutils.fs.ls("dbfs:/Volumes/taxi_eda_db/taxi_zone_lookups/data2/"))
# create dataframe
df_zone_lk = spark.read.csv("dbfs:/Volumes/taxi_eda_db/taxi_zone_lookups/data2/file_taxi_zone_lookup",
                            header=True,
                            inferSchema=True,
                            sep=",")

df_zone_lk.show(5)


+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows


In [0]:
df_PU = df_trips.select("PULocationID")
df_DO = df_trips.select("DOLocationID")
df_PU_DO = df_PU.union(df_DO)
df_borough = df_PU_DO.join(df_zone_lk, df_PU_DO.PULocationID == df_zone_lk.LocationID, "inner")
df_borough.show()

+------------+----------+---------+--------------------+------------+
|PULocationID|LocationID|  Borough|                Zone|service_zone|
+------------+----------+---------+--------------------+------------+
|         151|       151|Manhattan|    Manhattan Valley| Yellow Zone|
|         239|       239|Manhattan|Upper West Side S...| Yellow Zone|
|         236|       236|Manhattan|Upper East Side N...| Yellow Zone|
|         193|       193|   Queens|Queensbridge/Rave...|   Boro Zone|
|         193|       193|   Queens|Queensbridge/Rave...|   Boro Zone|
|         193|       193|   Queens|Queensbridge/Rave...|   Boro Zone|
|         193|       193|   Queens|Queensbridge/Rave...|   Boro Zone|
|         163|       163|Manhattan|       Midtown North| Yellow Zone|
|         229|       229|Manhattan|Sutton Place/Turt...| Yellow Zone|
|         141|       141|Manhattan|     Lenox Hill West| Yellow Zone|
|         246|       246|Manhattan|West Chelsea/Huds...| Yellow Zone|
|         238|      

# Manhattan has most pickup and dropoff

### Part 3

- choose 3 questions from above and re-answer them using the language you did not use for the main notebook . (i.e - if you completed the exercise in python, redo 3 questions in pure sql) . at least one of the questions to be redone must involve a join

#### Step 1 - Register the DataFrame

In [0]:
df_trips.createOrReplaceTempView("trips")

In [0]:
df_zone_lk.createOrReplaceTempView("taxi_zone_lookup")

#### Step 2 - Answer the questions in Spark SQL

##### 1. What is the Average passanger count

In [0]:
%sql
-- Calculate the average number of passengers per trip
SELECT 
    ROUND(AVG(passenger_count), 2) AS avg_passenger_count
FROM trips;

avg_passenger_count
1.57


On average, each taxi trip had about 1.57 passengers. This query calculates the mean passenger count across all trips. It gives us an idea of how many passengers are usually in one taxi ride.

##### 2. Busiest day/slowest single day

In [0]:
%sql
-- First query : busiest day
SELECT 
    'Busiest day' AS info, 
    DATE(tpep_pickup_datetime) AS trip_date, 
    COUNT(*) AS num_trips
FROM trips
GROUP BY DATE(tpep_pickup_datetime)
ORDER BY num_trips DESC
LIMIT 1;


info,trip_date,num_trips
Busiest day,2019-01-25,292499


The busiest day was January 25, 2019, with around 292,499 trips, showing a peak in taxi activity that day.

In [0]:
%sql
-- Second query: slowest day
SELECT 
    'Slowest day' AS info, 
    DATE(tpep_pickup_datetime) AS trip_date, 
    COUNT(*) AS num_trips
FROM trips
GROUP BY DATE(tpep_pickup_datetime)
ORDER BY num_trips ASC
LIMIT 1;

info,trip_date,num_trips
Slowest day,2018-12-21,1


The slowest day was December 21, 2018, with only 1 trip, likely due to missing data or an incomplete record before 2019.

##### 3. On average which day of the week is slowest/busiest

In [0]:
%sql
-- Number of trips per weekday
SELECT 
    DAYOFWEEK(tpep_pickup_datetime) AS day_of_week_num,
    DATE_FORMAT(tpep_pickup_datetime, 'EEEE') AS day_of_week,
    COUNT(*) AS num_trips
FROM trips
GROUP BY DAYOFWEEK(tpep_pickup_datetime), DATE_FORMAT(tpep_pickup_datetime, 'EEEE')
ORDER BY day_of_week_num ASC;


day_of_week_num,day_of_week,num_trips
1,Sunday,859905
2,Monday,908121
3,Tuesday,1209084
4,Wednesday,1265264
5,Thursday,1357043
6,Friday,1087215
7,Saturday,1009985


In [0]:
%sql
-- Busiest weekday
SELECT 
    DAYOFWEEK(tpep_pickup_datetime) AS day_of_week_num,
    DATE_FORMAT(tpep_pickup_datetime, 'EEEE') AS day_of_week,
    COUNT(*) AS num_trips
FROM trips
GROUP BY DAYOFWEEK(tpep_pickup_datetime), DATE_FORMAT(tpep_pickup_datetime, 'EEEE')
ORDER BY num_trips DESC
LIMIT 1;

day_of_week_num,day_of_week,num_trips
5,Thursday,1357043


In [0]:
%sql
-- Slowest weekday
SELECT 
    DAYOFWEEK(tpep_pickup_datetime) AS day_of_week_num,
    DATE_FORMAT(tpep_pickup_datetime, 'EEEE') AS day_of_week,
    COUNT(*) AS num_trips
FROM trips
GROUP BY DAYOFWEEK(tpep_pickup_datetime), DATE_FORMAT(tpep_pickup_datetime, 'EEEE')
ORDER BY num_trips ASC
LIMIT 1;

day_of_week_num,day_of_week,num_trips
1,Sunday,859905


- Thursday had the highest number of trips, making it the busiest day of the week.
- Sunday had the fewest trips, so it was the slowest day.

This suggests that weekdays, especially Thursday, are busier for taxis than weekends.


### Part 4

As of spark v4 dataframes have native visualization support. Choose at least 3 questions from above and provide visualizations.


# Where to go from here

- Continue building the dataset by loading in more data, start by completing the data for 2019 and calculating the busiest season (fall, winter, spring, summer)
- Explore a dataset/datasets of your choosing